In [1]:
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from random import sample
from datetime import datetime
import collections

!pip install -q tensorflow_addons
import tensorflow_addons as tfa

!pip install mne
import mne

!pip install wfdb pyEDFlib PyWavelets eeglib wfdb pyeeg

%matplotlib inline
import wfdb
import pyedflib
import pywt 
import eeglib

from mne import Epochs, create_info, events_from_annotations
from mne.io import concatenate_raws, read_raw_edf
from mne.decoding import CSP
from mne.time_frequency import AverageTFR
from mne.decoding import Vectorizer
from mne.io import concatenate_raws, read_raw_edf
from mne import Epochs, create_info, events_from_annotations, pick_events, pick_types

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     |████████████████████████████████| 1.7 MB 924 kB/s 
     |████████████████████████████████| 133 kB 72.0 MB/s 
     |████████████████████████████████| 195 kB 63.2 MB/s 
     |████████████████████████████████| 5.8 MB 47.3 MB/s 
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp37-cp37m-linux_x86_64.whl size=382853 sha256=a11377d798a0f22e20049941956527ed386148a1a411c1d8e8019996a4bebd92
  Stored in directory: /root/.cache/pip/wheels/a2/da/8c/2d1b9b233595056f05e59156bb555f7277b88beb385605de16
  Created wheel for websocket: filename=websocket-0.2.1-py3-none-any.whl size=192132 sha256=19099fc342d9688f806df81c6bd7f9580e64636b540895f2b3e85f91efa43bec
  Stored in directory: /root/.cache/pip/wheels/e2/9b/2b/6324f465ec5543e33a31c2692c01c1760173bbb98b37dbaee8
Successfully built fastdtw websocket
/kaggle/input/eeg-edfs-files-curated/seizures summary.csv
/kaggle/input/eeg-edfs-files-curated/edfs/chb17a_03.edf
/kaggle/input/eeg-edfs-files-curated/edfs/chb04_08.edf
/kaggle/input/eeg-edfs-files

In [2]:
# Cargamos la lista de todos los seizures y sus tiempos en cada fichero
df_seizures = pd.read_csv('/kaggle/input/eeg-edfs-files-curated/seizures summary.csv',delimiter=';', encoding='utf8')

In [3]:
from mne.io import concatenate_raws, read_raw_edf
from mne import Epochs, create_info, events_from_annotations, pick_events, pick_types

lista_erroneos = ['chb10_31.edf', 'chb04_05.edf','chb04_08.edf', 'chb09_19.edf', 'chb09_06.edf', 'chb06_18.edf','chb10_20.edf',
                  'chb10_38.edf', 'chb10_12.edf', 'chb23_06.edf', 'chb06_10.edf', 'chb06_24.edf', 'chb06_09.edf', 'chb07_12.edf', 
                  'chb10_27.edf']

df_seizures = pd.read_csv('/kaggle/input/eeg-edfs-files-curated/seizures summary.csv',delimiter=';', encoding='utf8')

df_seizures = df_seizures[~df_seizures['nombre archivo'].isin(lista_erroneos)]

def lee_raw(paciente):
    
    #elegidos=[]

    #for i in np.random.choice(df_seizures['paciente'].unique(), 1):
    #for i in paciente:
        #np.random.choice(df_seizures['paciente'].unique(), 1):
        #elegidos.append(np.random.choice(df_seizures[df_seizures['paciente'] == i]['nombre archivo']))
    #elegidos.append(np.random.choice(df_seizures[df_seizures['paciente'] == paciente]['nombre archivo'],1) )
    
    #elegidos.append(df_seizures[df_seizures['paciente'] == paciente]['nombre archivo'].sample().to_list()[0])
    
    elegido = df_seizures[df_seizures['paciente'] == paciente]['nombre archivo'].sample().to_list()[0]

    # Path with the recordings
    myPath = '/kaggle/input/eeg-edfs-files-curated/edfs/'
    # Ficheros totales (All patients)
    data = os.listdir(myPath)
    
    #epochs_all llevará todos los epochs de todos los pacientes que seleccionemos.
    epochs_all = np.empty(shape=())  # Empty array with size =  1
    data_all = np.empty(shape=()) # Empty array with size =  1
    epochs_all_10 = np.empty(shape=())  # Empty array with size =  1

    count = 0
    
    #print("Elegidos : {}".format(elegidos))
    #for i in elegidos:
        #print("type de i:{} and {}".format(type(i), i))
        #dataPath = os.path.join(myPath, data[i])
    
    dataPath = os.path.join(myPath, str(elegido))
        
    # We read our EDF recording and build a generic 'mapping' of channels that
    # we will remove since they are wrong or reference channels.
    mapping = {'EOG horizontal': 'eog', 'Resp oro-nasal': 'misc',
               'EMG submental': 'misc', 'Temp rectal': 'misc', 
               'Event marker': 'misc', '-': 'misc', '.': 'misc', 'T8-P8':'T8-P8',
               'FC1-Ref':'FC1-Ref', 'FC2-Ref':'FC2-Ref', 'FC5-Ref':'FC5-Ref',
               'FC6-Ref':'FC6-Ref', 'CP1-Ref':'CP1-Ref','CP2-Ref':'CP2-Ref',
               'CP5-Ref':'CP5-Ref', 'CP6-Ref':'CP6-Ref', 'PZ-OZ':'PZ-OZ'}
    
        # We will specifically exclude the channel with issues (T8-P8) and the weird channels (. and -)
    excluded_channels = mapping.keys()
        
    if str(elegido) in df_seizures['nombre archivo'].to_list():
        
        print("Leyendo fichero...:{}".format(elegido))
        print("Desde :{}".format(myPath+elegido))
        count +=1
        raw = mne.io.read_raw_edf(myPath+elegido, exclude=excluded_channels, preload=True, stim_channel='auto', verbose=None)
     
        raw = raw.resample(sfreq=127)

        #raw = mne.io.read_raw_edf(path, exclude=['-', 'T8-P8', '.'], verbose=False, preload=True)

        # Rename EEG channels
        ch_names = {i: i.replace('EEG ', '') for i in raw.ch_names if 'EEG' in i}
        mne.rename_channels(raw.info, ch_names)
        
        
        
        # We then apply notch filter around 60 Hz. (USA) or 50Hz (Europe) This is recommended by some papers to eliminate noise from electrodes (power line).
        raw = raw.notch_filter(freqs=[59.1, 60.9])

        # We then apply the alpha and beta filters that go from 7 to 26/28 Hz (FIRWIN type, default for MNE/Scipy)
        raw = raw.filter(l_freq=8.0, h_freq=29.0)

        # Now we mark the starts and ends of the seizures to obtain the class (Inter, Preictal, Ictal) for each second.
        seizure_init = float(df_seizures[df_seizures['nombre archivo']==elegido]['inicio seizure segundos'].to_list()[0])
        # print("seiz init", seizure_init)
        seizure_end = float(df_seizures[df_seizures['nombre archivo']==elegido]['fin seizure segundos'].to_list()[0])
        duration_seiz = float(seizure_end-seizure_init)
    
        preictal_init = float(seizure_init - 10*60)  # 10 minutos (se puede aumentar)
        preictal_duration = float(seizure_init - preictal_init)
        print("seiz_init, preictal_duration y seizure_end", seizure_init, preictal_duration, seizure_end)

        normal_antes_pre_dur = float(preictal_init)
        normal_despues_seiz_dur = float(3600 - seizure_end)

        this_file_annots = mne.Annotations(onset=[0, preictal_init, seizure_init, seizure_end ],  # in seconds
            duration=[normal_antes_pre_dur, preictal_duration, duration_seiz, normal_despues_seiz_dur],  # in seconds, too
            description=['Inter','Preictal','Ictal', 'Inter'],
            ch_names = None)
        
        raw.set_annotations(this_file_annots)
        
                
        # Recogemos los eventos de las anotaciones (para construir los futuros epochs a partir de eventos)
        # También podríamos usar duration=2.5 y overlap=0.5
        events, events_ids = mne.events_from_annotations(raw, event_id={'Preictal':0, 'Ictal':2, 'Inter':1}, 
                                                         chunk_duration=1, use_rounding=True )
        
        # Finally we crop the first minute to reduce noise (1 m. until patient settles)
        crop_file_mins = 1
        if crop_file_mins > 0:  # Cut start of file
            # Crop raw
            tmin_crop = crop_file_mins * 60
            #tmax = crop_file_mins * 60
            raw.crop(tmin=tmin_crop)
        
        # Save subject and recording information in raw.info
        # Guardamos la info del fichero: paciente/fichero.
        basename = elegido
        #print(int(basename[3:5]))

        subj_nb, rec_nb = str(basename[3:6]), str(basename[7:8])
        # print("Los integers", subj_nb, rec_nb)

        raw.info['patient_info'] = {'id': subj_nb, 'fichero_id': rec_nb}

        print('Patient_info info: {}'.format(raw.info['patient_info'])+'\n')

        if count == 1:

            raw_final = raw.get_data()

        else:

            raw_final = np.append(raw_final, raw.get_data(), axis=1)

        #mne.concatenate_epochs(epochs_list, add_offset=True, *, on_mismatch='raise', verbose=None)

        picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')

        # Jugando con tmin y tmax podemos obtener epochs de mas o menos duracion y/o solapamiento 
        epochs_all_tmp = mne.Epochs(raw, picks=picks, events=events, event_repeated = 'merge', 
                         event_id=events_ids, tmin=-0.0001, tmax=1.0, baseline =(None,None),
                         preload=True)
        
        # Construimos epochs (con el objeto Raw epochs_all_tmp de cada fichero).
        if data_all.size > 1:

            #epochs_all = np.append(epochs_all, epochs_all_tmp.get_data(), axis=0)

            epochs_PIN = epochs_all_tmp['Preictal', 'Ictal', 'Inter']
            final_df = final_df.append(epochs_PIN.to_data_frame())

            data_all_tmp = epochs_PIN.get_data()
            print('size of tmp : {}'.format(data_all_tmp.shape) +'\n')
            data_all = np.append(data_all, data_all_tmp, axis=0)

            #labels_data_tmp = epochs_all_tmp.events[:,-1]

            labels_data_all = np.append(labels_data_all, epochs_PIN.events[:,-1], axis = 0)

            #epochs_all_10 = np.append(epochs_all_10, epochs_all_tmp_10, axis=0)

        else:
            
            epochs_all = mne.Epochs(raw, picks=picks, events=events, event_repeated = 'merge', 
                         event_id=events_ids, tmin=-0.0001, tmax=1.0, baseline =(None,None),
                         preload=True)

            epochs_PIN = epochs_all['Preictal', 'Ictal', 'Inter']
                
            # Para 'unificar' clases por tamanos. 
            # epochs_PIN.equalize_event_counts(list(events_ids.keys()))
                
            final_df = epochs_PIN.to_data_frame()

            data_all = epochs_PIN.get_data()

            #print('size of dataall :{}'.format(data_all.shape) + '\n')
            labels_data_all = epochs_PIN.events[:,-1]

            #epochs_all_10 = epochs_all_tmp_10

#    else:

#        continue
    final_df['cat'] = [1 if i == 'Preictal' else 2 if i == 'Ictal' else 0 for i in final_df['condition']]
    
    return final_df

In [4]:
def compute_cwt(DF, wavelet, ventana=10, solapamiento=0, fs = 128):
    
    fs = fs
    final_df = DF
    waveletname = pywt.ContinuousWavelet(wavelet)
    escalas = pywt.scale2frequency(waveletname.name, np.arange(8,30.5,2)) / (1/fs)
    print("Numero de escalas que vamos a probar : {}".format(len(escalas)))
    ventana = ventana
    incremento = fs * ventana  # La ventana me da el incremento o 'avance' en cada iteracion
    solapamiento = solapamiento * fs
    
    train_size = int(np.floor(final_df.shape[0]/(ventana*fs)))
    
    n_channels = len(final_df.columns) - 4
    # Redondeamos para evitar el último epoch que desbordaría
    tamano_resultado_train = int(round(train_size,0))
    
    # Matriz resultado en train (tamaño muestras, escalas, freq.samp, canales)
    train_data_cwt = np.ndarray(shape=(tamano_resultado_train, len(escalas), fs, n_channels))
    train_data_cwt_labels = np.ndarray(shape=(tamano_resultado_train, 1, n_channels))
    
    for ii in range(0,tamano_resultado_train):
        if ii == int(np.floor(tamano_resultado_train/2)):
            print("CWT en set training a la mitad...{}".format(ii*incremento))
    
        for jj in range(0,n_channels):
        
            start = final_df['epoch'].min()
 
            signal = final_df.iloc[ii*(incremento - solapamiento):(ii+1)*(incremento-solapamiento), jj+3:jj+4]
    
            # Calculamos la clase con el 'techo' del valor de la mediana de cada ventana
            clase_intervalo = int(np.ceil(np.median(final_df.iloc[ii*(incremento- solapamiento) : (ii+1)*(incremento- solapamiento), -1])))
  
            train_data_cwt_labels[ii, : ,jj] = clase_intervalo
        
            coeff, freq = pywt.cwt(signal, escalas, waveletname, 1)
        
            coeff_approx = coeff[:,:fs]
        
            train_data_cwt[ii, :, :, jj] = abs(coeff_approx[:,:,-1])
    
    print("Final CWT decomp. \n") 
    
    return train_data_cwt, train_data_cwt_labels

In [5]:
fs = 128
escalas = pywt.scale2frequency('cmor3-3', np.arange(8,30.5,2) / (1/fs))
print("Numero de escalas que vamos a probar : {}".format(len(escalas)))

Numero de escalas que vamos a probar : 12


In [6]:
# Resultados individualizados

segundos = [5]
wavelets = ['morl', 'mexh']
solapamientos = [0]
resultados = dict()
#for i in df_seizures['paciente'].unique():
for i in np.random.choice(df_seizures['paciente'].unique(),8).tolist():
    temp = lee_raw(i)
    #lectura_edf = obtain_edf(i)
    for j in segundos:
        for k in wavelets:        
            resultados.update({'paciente_'+i+'_'+str(j)+'_'+k: temp})
            resultados.update({'paciente_'+i+'_'+str(j)+'_'+k: compute_cwt(temp, k, j, 0)})
            #numpy.save("npy.npy", data)

Leyendo fichero...:chb03_34.edf
Desde :/kaggle/input/eeg-edfs-files-curated/edfs/chb03_34.edf
Extracting EDF parameters from /kaggle/input/eeg-edfs-files-curated/edfs/chb03_34.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 839 samples (6.606 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 29 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband

/tmp/ipykernel_19/4092241418.py:98: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(this_file_annots)


61 bad epochs dropped
Not setting metadata
Not setting metadata
2990 matching events found
Setting baseline interval to [0.0, 1.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 2990 events and 128 original time points ...
61 bad epochs dropped
Numero de escalas que vamos a probar : 12
CWT en set training a la mitad...186880
Final CWT decomp. 

Numero de escalas que vamos a probar : 12
CWT en set training a la mitad...186880
Final CWT decomp. 

Leyendo fichero...:chb14_27.edf
Desde :/kaggle/input/eeg-edfs-files-curated/edfs/chb14_27.edf
Extracting EDF parameters from /kaggle/input/eeg-edfs-files-curated/edfs/chb14_27.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- 

In [7]:
#for k, v in resultados.items():
#    string_res_x = 'Data_'+ str(k)
#    string_res_y = 'Labels_'+ str(k)
#    np.save(string_res_x, v[0])
#    np.save(string_res_y, v[1])
    

[batch, timesteps, feature]

In [8]:
from tensorflow import keras
from keras.layers import Dense, Flatten, Dropout, LSTM
from keras.layers import Conv2D, MaxPooling2D, Conv3D, MaxPooling3D
from keras.models import Sequential
from keras.callbacks import History
#from keras.optimizers import SGD


num_escalas = list(resultados.values())[0][0].shape[1]
coeficientes = list(resultados.values())[0][0].shape[2]
canales = list(resultados.values())[0][0].shape[3]
input_shape = (num_escalas, coeficientes , canales)

num_classes = 3
batch_size = 1
epochs = 50


model = Sequential()
model.add(LSTM(64, input_shape=(input_shape[1], input_shape[0]*input_shape[2]), return_sequences=True,  activation="tanh",
    recurrent_activation="sigmoid"))
model.add(Dropout(0.2))
model.add(LSTM(32,activation="tanh", recurrent_activation="sigmoid", return_sequences=False))
model.add(Dropout(0.2))
#model.add(LSTM(100,return_sequences=True))
#model.add(Dropout(0.2))
#model.add(LSTM(50))
#model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

    
METRICS = ['accuracy',
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'),
      tfa.metrics.F1Score(name= 'f1_score', num_classes=3, average = 'macro'),
      #tfa.metrics.F1Score(num_classes = 6, average = "macro",name = "f1_score")
      tf.keras.metrics.CategoricalCrossentropy(name='categorical_crossentropy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
                metrics=['accuracy',
                tfa.metrics.F1Score(name= 'f1_score', num_classes=3),
                tf.keras.metrics.CategoricalCrossentropy(name='categorical_crossentropy'),
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall')])


#model.compile(loss=keras.losses.categorical_crossentropy,
#              optimizer=keras.optimizers.Adam(learning_rate=0.001),
#                metrics= METRICS)

model.summary()


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_P

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128, 64)           81152     
_________________________________________________________________
dropout (Dropout)            (None, 128, 64)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 3)                 99        
Total params: 93,667
Trainable params: 93,667
Non-trainable params: 0
_________________________________________________________________


In [9]:
def create_callbacks(model, metric = 'val_accuracy'):
    
    #guardar_path = '/home/inaki/temp/mejor_modelo_'+ model.name + '.h5'
    
    #checkpoint = tf.keras.callbacks.ModelCheckpoint(
    #    filepath = guardar_path,
    #    monitor = metric,
    #    mode ='max',
    #    save_best_only=True,
    #    verbose=1,
    #)
    
    reduccionlr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor = metric,
        mode = 'max',
        factor = 0.1,
        patience = 3,
        verbose = 0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor = metric,
        mode ='max',
        patience = 5, 
        verbose = 1
    )
    
    callbacks = [reduccionlr, earlystop]         
    
    return callbacks

In [10]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor = 'val_f1_score', patience=5, mode='max', verbose = 1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_f1_score',  factor = 0.1, patience = 3, verbose =0),
]

In [11]:
def process_model(data, labels):
    from random import sample
    
    train_data_cwt = data
    train_data_cwt_labels = labels
    
    #print(train_data_cwt.shape)
    #print(train_data_cwt_labels.shape)
    
    items = [item for sublist in train_data_cwt_labels[:,:,-1] for item in sublist]

    import collections

    # using Counter from Dict() to find frequency of elements
    balanceo = collections.Counter(items)
    suma = sum(balanceo.values())
    for k, v in balanceo.items():
        balanceo[k] = (1/v)*(suma/3)
    print("Balanceo clases : {}".format(dict(balanceo)))

    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler()
    #num_instances, escalas, coef, num_canales = train_data_cwt.shape
    train_data = np.reshape(train_data_cwt, (-1, train_data_cwt.shape[-1]))
    train_data = scaler.fit_transform(train_data)
    train_data = np.reshape(train_data, (train_data_cwt.shape[0], train_data_cwt.shape[1], train_data_cwt.shape[2], train_data_cwt.shape[-1]))
    
    random_num_generated = int(sample(range(0,99), 1)[0])
    
    # Ensure that we get at least one sample of each class on both train and test
    X_train, X_test, y_train, y_test = train_test_split(train_data, train_data_cwt_labels,  test_size=0.3, random_state = random_num_generated)
    #print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    
    y_train_cat = tf.keras.utils.to_categorical(y_train[:,-1,-1], 3)
    y_test_cat = tf.keras.utils.to_categorical(y_test[:,-1,-1], 3)
    
    #print("Shapes", y_train_cat.shape, y_test_cat.shape)
    
    # Ensure that we get at least one sample of each class on both train and test
    while len(set(np.argmax(y_test_cat, axis=1).tolist())) < len(set(np.argmax(y_train_cat, axis=1).tolist())):
        random_num_generated = int(sample(range(0,99), 1)[0])
        X_train, X_test, y_train, y_test = train_test_split(train_data, train_data_cwt_labels,  test_size=0.3, random_state = random_num_generated)
        y_train_cat = tf.keras.utils.to_categorical(y_train[:,-1,-1], 3)
        y_test_cat = tf.keras.utils.to_categorical(y_test[:,-1,-1], 3)
    
    print("Y shapes before", y_train_cat.shape, y_test_cat.shape)
    print("X shapes before", X_train.shape, X_test.shape)
    
    X_train = X_train.reshape(X_train.shape[0], 128, -1)
    X_test = X_test.reshape(X_test.shape[0], 128, -1)
    
    y_train_cat = y_train_cat.reshape(y_train_cat.shape[0], -1)
    y_test_cat = y_test_cat.reshape(y_test_cat.shape[0], -1)
    
    print("Y shapes AFTER", y_train_cat.shape, y_test_cat.shape)
    print("X shapes AFTER", X_train.shape, X_test.shape)
    
    batch_size = 1
    epochs = 30
    
    
    METRICS = ['accuracy',
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'),
      tfa.metrics.F1Score(name= 'f1_score', num_classes=3),
      tf.keras.metrics.CategoricalCrossentropy(name='categorical_crossentropy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]
    
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics= METRICS)
    
    history = History()
    history = model.fit(X_train, y_train_cat,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test_cat),
          #callbacks = my_callbacks,
          callbacks = [history],
          class_weight = balanceo)

    ultimos = dict()
    
    for k, v in history.history.items():
        ultimos.update({k: v[-1]})  
    #return X_train, X_test, y_train_cat, y_test_cat
    return ultimos

Operamos ahora con los resultados de todos los CWTs obtenidos por paciente, ejecutando el modelo LSTM para cada uno de los pacientes y guardando los resultados.

In [12]:
historias = dict()
for k, v in resultados.items():
    #a = v[0]
    #b = v[1]
    #print(a.shape, b.shape)
    hist_temp = process_model(v[0], v[1])
    historias.update({k:hist_temp})

Balanceo clases : {0.0: 0.40843443096476023, 1.0: 1.9638888888888888, 2.0: 23.566666666666666}
Y shapes before (494, 3) (213, 3)
X shapes before (494, 12, 128, 21) (213, 12, 128, 21)
Y shapes AFTER (494, 3) (213, 3)
X shapes AFTER (494, 128, 252) (213, 128, 252)


2021-12-08 05:02:51.028007: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30
494/494 [==============================] - 37s 65ms/step - loss: 1.1561 - accuracy: 0.7024 - tp: 313.0000 - fp: 117.0000 - tn: 871.0000 - fn: 181.0000 - f1_score: 0.4663 - categorical_crossentropy: 0.6988 - precision: 0.7279 - recall: 0.6336 - auc: 0.8613 - prc: 0.7397 - val_loss: 0.8098 - val_accuracy: 0.6291 - val_tp: 131.0000 - val_fp: 75.0000 - val_tn: 351.0000 - val_fn: 82.0000 - val_f1_score: 0.4510 - val_categorical_crossentropy: 0.8098 - val_precision: 0.6359 - val_recall: 0.6150 - val_auc: 0.7854 - val_prc: 0.5514
Epoch 2/30
494/494 [==============================] - 29s 59ms/step - loss: 0.6138 - accuracy: 0.6296 - tp: 300.0000 - fp: 176.0000 - tn: 812.0000 - fn: 194.0000 - f1_score: 0.4735 - categorical_crossentropy: 0.7909 - precision: 0.6303 - recall: 0.6073 - auc: 0.8158 - prc: 0.6336 - val_loss: 0.5743 - val_accuracy: 0.7512 - val_tp: 156.0000 - val_fp: 50.0000 - val_tn: 376.0000 - val_fn: 57.0000 - val_f1_score: 0.5587 - val_categorical_crossentropy: 0.5743 -

In [13]:
from datetime import datetime
historias_df = pd.DataFrame(historias)
now = datetime.now()
dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
historias_df.to_csv('/kaggle/working/historias_df_LSTM'+dt_string+'.csv', sep=',', na_rep='', header=True, index=True, mode='w')